In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import localSettings as ls
print(ls.main_path)

/Users/elenag/Desktop/PlotterLEE/PELEE3/


In [3]:
main_path = ls.main_path
sys.path.append(main_path)

In [4]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

date and time: 11042020


In [5]:
SHRCALIBFACTOR = 1.00

In [6]:
# USING BDT?
USEBDT = True

In [7]:
# Doing Run3?
ISRUN3 = False

In [8]:
# scale to MCC8 CV?
MCC8WEIGHTS = False

In [9]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
tree = "NeutrinoSelectionFilter"

BNB = 'neutrinoselection_filt_run1_beamon_beamgood_all' +ls.APPEND
EXT = 'neutrinoselection_filt_run1_beamoff_all'         +ls.APPEND
NU  = 'neutrinoselection_filt_run1_overlay'         +ls.APPEND
NUE = 'neutrinoselection_filt_run1_overlay'         +ls.APPEND
DRT = 'neutrinoselection_filt_run1_dirt_overlay'    +ls.APPEND

mc   = uproot.open(ls.ntuple_path+NU +".root")[ls.fold][tree]
nue  = uproot.open(ls.ntuple_path+NUE+".root")[ls.fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[ls.fold][tree]
ext  = uproot.open(ls.ntuple_path+EXT+".root")[ls.fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[ls.fold][tree]
#lee = uproot.open(ls.ntuple_path+NUE+".root")[ls.fold][tree]

#uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,cccpi,ncnopi,nccpi,nue,ext,data,dirt]
uproot_v = [mc,nue,ext,data,dirt]
#uproot_v = [ext,data]


variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "flash_time",
    "shr_tkfit_dedx_U","shr_tkfit_dedx_V","shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    "topological_score",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP","CosmicDirAll3D","CosmicIPAll3D",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    "_opfilter_pe_beam", "_opfilter_pe_veto", # did the event pass the common optical filter (for MC only)
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_nhits_Y","shr_tkfit_nhits_U","shr_tkfit_nhits_V",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    "pfnhits","pfnunhits",
    "trk_sce_start_x_v","trk_sce_start_y_v","trk_sce_start_z_v",
    "trk_sce_end_x_v","trk_sce_end_y_v","trk_sce_end_z_v","shr_id", 
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U",
    "pi0_dir2_x","pi0_dir2_y","pi0_dir2_z","pi0_dir1_x","pi0_dir1_y","pi0_dir1_z",
    "pi0truth_gamma1_etot","pi0truth_gamma2_etot","pi0truth_gammadot","pi0truth_gamma_parent",
    "trk_theta_v","trk_phi_v"
]
#make the list unique
variables = list(set(variables))
print(variables)

#if not ISRUN3:
variables.remove("_closestNuCosmicDist")
variables.remove("crtveto")
variables.remove("crthitpe")
    
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune"]#, "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight"]#, "weightsGenie", "weightsFlux", "weightsReint"]
MCFVARS = ["mcf_nu_e","mcf_lep_e","mcf_actvol","mcf_nmm","mcf_nmp","mcf_nem","mcf_nep","mcf_np0","mcf_npp",
           "mcf_npm","mcf_mcshr_elec_etot","mcf_pass_ccpi0","mcf_pass_ncpi0",
           "mcf_pass_ccnopi","mcf_pass_ncnopi","mcf_pass_cccpi","mcf_pass_nccpi"]
SYSTEMATICS = ['weightsFlux','weightsGenie']

nue = nue.pandas.df(variables + WEIGHTS + SYSTEMATICS, flatten=False)
mc = mc.pandas.df(variables + WEIGHTS + MCFVARS + SYSTEMATICS, flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + WEIGHTS + SYSTEMATICS, flatten=False)


['pi0_dot2', 'shr_tkfit_2cm_nhits_U', 'trk_score', 'shr_score', 'topological_score', 'reco_nu_vtx_sce_z', 'shr_dedx_Y', 'pi0_dedx1_fit_U', 'reco_nu_vtx_sce_x', 'shr_tkfit_dedx_U', 'pi0_dir1_x', 'trk_hits_tot', 'slpdg', 'sub', 'shrsubclusters0', 'trk_energy_proton_v', 'ccnc', 'shrclusfrac0', 'slclustfrac', 'trk_sce_end_y_v', 'pi0_dedx2_fit_V', 'interaction', 'trk_energy', 'pfnhits', 'nproton', 'pi0_mass_V', 'shr_tkfit_npoints', 'shrclusfrac1', 'shrsubclusters2', 'shr_tkfit_gap10_nhits_U', 'shr_tkfit_dedx_V', 'shr_distance', 'shr_energy', 'pi0truth_elec_etot', 'nu_e', 'trk_len', 'slnunhits', 'category', 'pi0_dedx2_fit_Y', 'shrmoliererms', 'run', 'evt', 'shr_tkfit_npointsvalid', 'pi0_dot1', 'pi0_dedx2_fit_U', 'reco_nu_vtx_y', 'shr_tkfit_dedx_Y', '_opfilter_pe_beam', 'shrclusdir2', 'pi0_radlen1', 'selected', 'pi0_mass_Y', 'shr_tkfit_2cm_dedx_V', 'pt', 'npi0', 'shr_tkfit_nhits_U', 'shr_bkt_pdg', 'hits_y', 'trk_sce_start_y_v', 'matched_E', 'shr_dedx_V', 'pi0_shrscore1', 'backtracked_pdg', 's

In [ ]:
df_v = [mc,nue,dirt]
#removeGenieTune = False
for i,df in enumerate(df_v):
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.    
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #if removeGenieTune : 
    #    df[ 'weightTune' ] = 1.
    #    df[ 'weightSplineTimesTune' ] = df['weightSpline']

    #df['weightSpline']  = df['weightSpline']  * df['weightTune']
    #df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] = df['weightSpline'] * df['weightTune'] * 0.7 #scale down pi0s

In [ ]:
df_v = [mc,nue,dirt]
#removeGenieTune = False
for i,df in enumerate(df_v):
    df[ 'currentweightsafe' ] = df['weightSplineTimesTune']
    df[ 'currentweight' ] = df['weightSplineTimesTune']

In [ ]:

scaleFlat = 2
#removeGenieTune = False
for i,df in enumerate(df_v):
    if scaleFlat == 1:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 
        df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] =  (df['weightSplineTimesTune'] * 0.759)
    elif scaleFlat == 2:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 
        pi0emax = 0.6
        df.loc[ (df['pi0_e'] > 0.1) & (df['pi0_e'] < pi0emax) , 'weightSplineTimesTune'] = df['weightSplineTimesTune']*(1.-0.4*df['pi0_e'])
        df.loc[ (df['pi0_e'] > 0.1) & (df['pi0_e'] >= pi0emax), 'weightSplineTimesTune'] = df['weightSplineTimesTune']*(1.-0.4*pi0emax)
    elif scaleFlat == 0:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!
import time
s = time.time()
df_v = [mc,nue,ext,data,dirt]
print(data.shape)
print(ext.shape)
print(nue.shape)
print(mc.shape)
puppa1 = 'trk_score_v'
#print ('trk_llr_pid_score_v' in nue.columns.to_list())
for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_score_v         = up.array('trk_score_v')    
    trk_llr_pid_v       = up.array('trk_llr_pid_score_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_id              = up.array('trk_id')-1 # I think we need this -1 to get the right result
    c                   = up.array('n_tracks_contained')
    trk_theta_proton_v  = up.array('trk_theta_v')
    trk_phi_proton_v    = up.array('trk_phi_v')

    print(trk_score_v.shape, len(trk_llr_pid_v))
#    print(len(c), len(trk_id), len(trk_llr_pid_v))
#    for pidv,puppa,tid,cid in zip(trk_llr_pid_v,trk_score_puppa_v,trk_id,c):
#        if len(pidv) > 0:
#            print(len(pidv),len(puppa), pidv,puppa,tid,cid )
#    nnnnnnnPupppppppaaa = 0
#    print(type(trk_llr_pid_v), trk_llr_pid_v.shape)
    puppa_obj = []
    puppa_trk = []
    puppa_shw = []
    puppa_prt = [] 
    puppa_mu  = []
    
    for pidv, p in zip(trk_llr_pid_v,trk_score_v):
        N_obj         = 0
        N_recoTrks    = 0
        N_recoShowers = 0
        N_recoProtons = 0
        N_recoMuons   = 0
        if len(pidv) > 0:
            passPIDCut = (pidv <= 0)
            passTrkCut = (p    >= 0)
            protonCut  = (passPIDCut & passTrkCut) 
            muonCut    = (np.logical_not(passPIDCut) & passTrkCut) 
            N_obj         = len(passTrkCut)
            N_recoTrks    = np.count_nonzero(passTrkCut)
            N_recoShowers = np.count_nonzero(np.logical_not(passTrkCut))
            N_recoProtons = np.count_nonzero(protonCut)
            N_recoMuons   = np.count_nonzero(muonCut)
        puppa_obj.append(N_obj)
        puppa_trk.append(N_recoTrks)
        puppa_shw.append(N_recoShowers)    
        puppa_prt.append(N_recoProtons)
        puppa_mu .append(N_recoMuons)

    df['N_obj']           = puppa_obj
    df['N_recoTrks']      = puppa_trk
    df['N_recoShowers']   = puppa_shw
    df['N_recoProtons']   = puppa_prt
    df['N_recoMuons']     = puppa_mu
    
    trk_llr_pid_v_sel     = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    trk_theta_proton_sel  = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_theta_proton_v,trk_id)])
    trk_phi_proton_sel    = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_phi_proton_v,trk_id)])

    #df['n_proton_cand']   = puppa_proton
    df['trkpid']          = trk_llr_pid_v_sel
    df['protonenergy']    = trk_energy_proton_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['subcluster']      = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit']          = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y']     = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    df['trkthetacos']     = np.cos(df['trk_theta'])

    df['protontheta']  = trk_theta_proton_sel
    df['protonphi']    = trk_phi_proton_sel


    #print(  df['N_recoShowers'])
    
s2 = time.time()
print (s-s2)

In [ ]:
print(data['N_recoProtons'])

In [ ]:
def get_elm_from_vec_idx(myvec,idx,defval=9999.):
    return awkward.fromiter([pidv[tid] if tid<len(pidv) else defval for pidv,tid in zip(myvec,idx)])
# this function returns the index in a vector at position argidx after it has been masked and sorted
# the returned index refers to the original (unsorted and unmaksed) vector
def get_idx_from_vec_sort(argidx,vecsort,mask):
    vid = vecsort[mask]
    sizecheck = argidx if argidx>=0 else abs(argidx)-1
    # find the position in the array after masking
    mskd_pos = [v.argsort()[argidx] if len(v)>sizecheck else -1 for v in vid]
    # go back to the corresponding position in the origin array before masking
    result = [[i for i, n in enumerate(m) if n == 1][p] if (p)>=0 else -1 for m,p in zip(mask,mskd_pos)]
    return result

def distance(x1,y1,z1,x2,y2,z2):
    return np.sqrt( (x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2 )

def cosAngleTwoVecs(vx1,vy1,vz1,vx2,vy2,vz2):
    return (vx1*vx2 + vy1*vy2 + vz1*vz2)/(np.sqrt(vx1**2+vy1**2+vz1**2) * np.sqrt(vx2**2+vy2**2+vz2**2))

def pick_closest_shower(up,df):
    #
    trk1_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    shr1_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    #
    # do the best we can to get the right shr2_id
    #
    trk_score_v = up.array("trk_score_v")
    shr_mask = (trk_score_v<0.5)
    pfnhits_v = up.array("pfnhits")
    shr2_id_corr = up.array('shr2_id')-1 # I think we need this -1 to get the right result
    shr2_id_appr = get_idx_from_vec_sort(-2,pfnhits_v,shr_mask)
    shr2_id = np.where((shr2_id_corr>=0)&(shr2_id_corr<df['n_showers_tot']),shr2_id_corr,shr2_id_appr)
    #
    shr_start_x_v   = up.array("shr_start_x_v")
    shr_start_y_v   = up.array("shr_start_y_v")
    shr_start_z_v   = up.array("shr_start_z_v")
    trk_start_x_v   = up.array("trk_start_x_v")
    trk_start_y_v   = up.array("trk_start_y_v")
    trk_start_z_v   = up.array("trk_start_z_v")
    #
    df["shr1_start_x"] = get_elm_from_vec_idx(shr_start_x_v,shr1_id,-9999.)
    df["shr1_start_y"] = get_elm_from_vec_idx(shr_start_y_v,shr1_id,-9999.)
    df["shr1_start_z"] = get_elm_from_vec_idx(shr_start_z_v,shr1_id,-9999.)
    df["shr2_start_x"] = get_elm_from_vec_idx(shr_start_x_v,shr2_id,-9999.)
    df["shr2_start_y"] = get_elm_from_vec_idx(shr_start_y_v,shr2_id,-9999.)
    df["shr2_start_z"] = get_elm_from_vec_idx(shr_start_z_v,shr2_id,-9999.)
    df["trk1_start_x"] = get_elm_from_vec_idx(trk_start_x_v,trk1_id,-9999.)
    df["trk1_start_y"] = get_elm_from_vec_idx(trk_start_y_v,trk1_id,-9999.)
    df["trk1_start_z"] = get_elm_from_vec_idx(trk_start_z_v,trk1_id,-9999.)
    #
    df['tk1sh1_distance'] = np.where((df['n_showers_contained']>0)&(df['n_tracks_contained']>0),\
                                     distance(df['shr1_start_x'],df['shr1_start_y'],df['shr1_start_z'],\
                                              df['trk1_start_x'],df['trk1_start_y'],df['trk1_start_z']),\
                                     9999.)
    df['tk1sh2_distance'] = np.where((df['n_showers_contained']>1)&(df['n_tracks_contained']>0),\
                                     distance(df['shr2_start_x'],df['shr2_start_y'],df['shr2_start_z'],\
                                              df['trk1_start_x'],df['trk1_start_y'],df['trk1_start_z']),\
                                     9999.)
    # set the shr_id
    df['shr_id'] = shr1_id
    df["is_shr2clsr"] = np.zeros_like(df["n_tracks_contained"])
    shr2clsr = (df['n_showers_contained']>1)&(df['n_tracks_contained']>0)&(df['tk1sh2_distance']<df['tk1sh1_distance'])
    df.loc[shr2clsr, 'is_shr2clsr' ] = 1
    #
    # now redefine shower selection variables
    # shr_score
    df["shr2_score"] = get_elm_from_vec_idx(trk_score_v,shr2_id,-9999.)
    df.loc[shr2clsr,"shr_score"] = df["shr2_score"]
    # tksh_distance
    df.loc[shr2clsr,"tksh_distance"] = df['tk1sh2_distance']
    # tksh_angle
    shr_px_v = up.array("shr_px_v")
    shr_py_v = up.array("shr_py_v")
    shr_pz_v = up.array("shr_pz_v")
    df["shr2_px"] = get_elm_from_vec_idx(shr_px_v,shr2_id,-9999.)
    df["shr2_py"] = get_elm_from_vec_idx(shr_py_v,shr2_id,-9999.)
    df["shr2_pz"] = get_elm_from_vec_idx(shr_pz_v,shr2_id,-9999.)
    trk_dir_x_v = up.array("trk_dir_x_v")
    trk_dir_y_v = up.array("trk_dir_y_v")
    trk_dir_z_v = up.array("trk_dir_z_v")
    df["trk1_dir_x"] = get_elm_from_vec_idx(trk_dir_x_v,trk1_id,-9999.)
    df["trk1_dir_y"] = get_elm_from_vec_idx(trk_dir_y_v,trk1_id,-9999.)
    df["trk1_dir_z"] = get_elm_from_vec_idx(trk_dir_z_v,trk1_id,-9999.)
    df["tk1sh2_angle"] = cosAngleTwoVecs(df["trk1_dir_x"],df["trk1_dir_y"],df["trk1_dir_z"],\
                                         df["shr2_px"],    df["shr2_py"],    df["shr2_pz"])
    df.loc[shr2clsr,"tksh_angle"] = df['tk1sh2_angle']
    # shr_tkfit_dedx_max
    shr_tkfit_dedx_u_v = up.array("shr_tkfit_dedx_u_v")
    shr_tkfit_dedx_v_v = up.array("shr_tkfit_dedx_v_v")
    shr_tkfit_dedx_y_v = up.array("shr_tkfit_dedx_y_v")
    shr_tkfit_nhits_u_v = up.array("shr_tkfit_dedx_nhits_u_v")
    shr_tkfit_nhits_v_v = up.array("shr_tkfit_dedx_nhits_v_v")
    shr_tkfit_nhits_y_v = up.array("shr_tkfit_dedx_nhits_y_v")
    df["shr2_tkfit_dedx_u"] = get_elm_from_vec_idx(shr_tkfit_dedx_u_v,shr2_id,-9999.)
    df["shr2_tkfit_dedx_v"] = get_elm_from_vec_idx(shr_tkfit_dedx_v_v,shr2_id,-9999.)
    df["shr2_tkfit_dedx_y"] = get_elm_from_vec_idx(shr_tkfit_dedx_y_v,shr2_id,-9999.)
    df["shr2_tkfit_nhits_u"] = get_elm_from_vec_idx(shr_tkfit_nhits_u_v,shr2_id,0)
    df["shr2_tkfit_nhits_v"] = get_elm_from_vec_idx(shr_tkfit_nhits_v_v,shr2_id,0)
    df["shr2_tkfit_nhits_y"] = get_elm_from_vec_idx(shr_tkfit_nhits_y_v,shr2_id,0)
    df.loc[shr2clsr, 'shr_tkfit_dedx_U' ] = df["shr2_tkfit_dedx_u"]
    df.loc[shr2clsr, 'shr_tkfit_dedx_V' ] = df["shr2_tkfit_dedx_v"]
    df.loc[shr2clsr, 'shr_tkfit_dedx_Y' ] = df["shr2_tkfit_dedx_y"]
    df.loc[shr2clsr, 'shr_tkfit_nhits_U' ] = df['shr2_tkfit_nhits_u']
    df.loc[shr2clsr, 'shr_tkfit_nhits_V' ] = df['shr2_tkfit_nhits_v']
    df.loc[shr2clsr, 'shr_tkfit_nhits_Y' ] = df['shr2_tkfit_nhits_y']
    # trkfit
    shr_tkfit_nhits_v = up.array("shr_tkfit_nhits_v")
    df["shr2_tkfit_npointsvalid"] = get_elm_from_vec_idx(shr_tkfit_nhits_v,shr2_id,-9999.)
    df["shr2_tkfit_npoints"] = get_elm_from_vec_idx(pfnhits_v,shr2_id,-9999.)
    df.loc[shr2clsr,"shr_tkfit_npointsvalid"] = df["shr2_tkfit_npointsvalid"]
    df.loc[shr2clsr, 'shr_tkfit_npoints' ] = df["shr2_tkfit_npoints"]
    # subcluster
    pfpplanesubclusters_U_v = up.array("pfpplanesubclusters_U")
    pfpplanesubclusters_V_v = up.array("pfpplanesubclusters_V")
    pfpplanesubclusters_Y_v = up.array("pfpplanesubclusters_Y")
    df["shr2subclusters0"] = get_elm_from_vec_idx(pfpplanesubclusters_U_v,shr2_id,0)
    df["shr2subclusters1"] = get_elm_from_vec_idx(pfpplanesubclusters_V_v,shr2_id,0)
    df["shr2subclusters2"] = get_elm_from_vec_idx(pfpplanesubclusters_Y_v,shr2_id,0)
    df.loc[shr2clsr,"shrsubclusters0"] = df["shr2subclusters0"]
    df.loc[shr2clsr,"shrsubclusters1"] = df["shr2subclusters1"]
    df.loc[shr2clsr,"shrsubclusters2"] = df["shr2subclusters2"]
    # shrmoliereavg
    shr_moliere_avg_v = up.array("shr_moliere_avg_v")
    df["shr2_moliere_avg"] = get_elm_from_vec_idx(shr_moliere_avg_v,shr2_id,-9999.)
    df.loc[shr2clsr,"shrmoliereavg"] = df['shr2_moliere_avg']
    # trkshrhitdist2
    df.loc[shr2clsr,"trkshrhitdist2"] = df['tksh_distance']
    #
    return


def process_uproot(up,df):
    #
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    #
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    trk_energy_proton_v = up.array('trk_energy_proton_v')
    trk_theta_v         = up.array('trk_theta_v')
    trk_phi_v           = up.array('trk_phi_v')
    trk_theta_sel         = get_elm_from_vec_idx(trk_theta_v,trk_id)
    trk_phi_sel           = get_elm_from_vec_idx(trk_phi_v,trk_id)
    df['protontheta']     = trk_theta_sel
    df['protonphi']       = trk_phi_sel

    #
    trk_llr_pid_v_sel     = get_elm_from_vec_idx(trk_llr_pid_v,trk_id)
    trk_calo_energy_y_sel = get_elm_from_vec_idx(trk_calo_energy_y_v,trk_id)
    trk_energy_proton_sel = get_elm_from_vec_idx(trk_energy_proton_v,trk_id)
    
    df['trkpid']          = trk_llr_pid_v_sel
    df['trackcaloenergy'] = trk_calo_energy_y_sel
    df['protonenergy']    = trk_energy_proton_sel
    trk_sce_start_x_v = up.array('trk_sce_start_x_v')
    trk_sce_start_y_v = up.array('trk_sce_start_y_v')
    trk_sce_start_z_v = up.array('trk_sce_start_z_v')
    trk_sce_end_x_v = up.array('trk_sce_end_x_v')
    trk_sce_end_y_v = up.array('trk_sce_end_y_v')
    trk_sce_end_z_v = up.array('trk_sce_end_z_v')
    df['shr_trk_sce_start_x'] = get_elm_from_vec_idx(trk_sce_start_x_v,shr_id)
    df['shr_trk_sce_start_y'] = get_elm_from_vec_idx(trk_sce_start_y_v,shr_id)
    df['shr_trk_sce_start_z'] = get_elm_from_vec_idx(trk_sce_start_z_v,shr_id)
    df['shr_trk_sce_end_x'] = get_elm_from_vec_idx(trk_sce_end_x_v,shr_id)
    df['shr_trk_sce_end_y'] = get_elm_from_vec_idx(trk_sce_end_y_v,shr_id)
    df['shr_trk_sce_end_z'] = get_elm_from_vec_idx(trk_sce_end_z_v,shr_id)
    df['shr_trk_len'] = distance(df['shr_trk_sce_start_x'],df['shr_trk_sce_start_y'],df['shr_trk_sce_start_z'], \
                                 df['shr_trk_sce_end_x'],  df['shr_trk_sce_end_y'],  df['shr_trk_sce_end_z'])
    df['mevcm'] = 1000 * df['shr_energy_tot_cali'] / df['shr_trk_len']
    #
    df["slclnhits"] = up.array("pfnhits").sum()
    df["slclnunhits"] = up.array("pfnunhits").sum()
    #
    trk_score_v = up.array("trk_score_v")
    shr_mask = (trk_score_v<0.5)
    trk_mask = (trk_score_v>0.5)
    df['n_tracks_tot'] = trk_mask.sum()
    df['n_showers_tot'] = shr_mask.sum()
    trk_len_v = up.array("trk_len_v")
    df["n_trks_gt10cm"] = (trk_len_v[trk_mask>=0.5]>10).sum()
    df["n_trks_gt25cm"] = (trk_len_v[trk_mask>=0.5]>25).sum()
    #
    pfnhits_v = up.array("pfnhits")
    trk_id_all = get_idx_from_vec_sort(-1,pfnhits_v,trk_mask) # this includes also uncontained tracks
    #
    shr_start_x_v   = up.array("shr_start_x_v")
    shr_start_y_v   = up.array("shr_start_y_v")
    shr_start_z_v   = up.array("shr_start_z_v")
    df["shr_start_x"] = get_elm_from_vec_idx(shr_start_x_v,shr_id)
    df["shr_start_y"] = get_elm_from_vec_idx(shr_start_y_v,shr_id)
    df["shr_start_z"] = get_elm_from_vec_idx(shr_start_z_v,shr_id)
    trk_start_x_v   = up.array("trk_start_x_v")
    trk_start_y_v   = up.array("trk_start_y_v")
    trk_start_z_v   = up.array("trk_start_z_v")
    df["trk1_start_x_alltk"] = get_elm_from_vec_idx(trk_start_x_v,trk_id_all)
    df["trk1_start_y_alltk"] = get_elm_from_vec_idx(trk_start_y_v,trk_id_all)
    df["trk1_start_z_alltk"] = get_elm_from_vec_idx(trk_start_z_v,trk_id_all)
    trk_dir_x_v = up.array("trk_dir_x_v")
    trk_dir_y_v = up.array("trk_dir_y_v")
    trk_dir_z_v = up.array("trk_dir_z_v")
    df["trk1_dir_x_alltk"] = get_elm_from_vec_idx(trk_dir_x_v,trk_id_all)
    df["trk1_dir_y_alltk"] = get_elm_from_vec_idx(trk_dir_y_v,trk_id_all)
    df["trk1_dir_z_alltk"] = get_elm_from_vec_idx(trk_dir_z_v,trk_id_all)
    #
    # tksh_distance and tksh_angle for track with most hits, regardless of containment
    #
    df['tk1sh1_distance_alltk'] = np.where(df['n_tracks_tot']==0,99999,
                                     distance(df['shr_start_x'],       df['shr_start_y'],       df['shr_start_z'],\
                                              df['trk1_start_x_alltk'],df['trk1_start_y_alltk'],df['trk1_start_z_alltk']))
    df["tk1sh1_angle_alltk"] = np.where(df['n_tracks_tot']==0,99999,
                                  cosAngleTwoVecs(df["trk1_dir_x_alltk"],df["trk1_dir_y_alltk"],df["trk1_dir_z_alltk"],\
                                                  df["shr_px"],          df["shr_py"],          df["shr_pz"]))
    #
    # fix the 'subcluster' bug (in case of more than one shower, it comes from the one with least hits, not the one with most)
    # so we overwrite the dataframe column taking the correct value from the corrsponding vector branches
    #
    pfpplanesubclusters_U_v = up.array("pfpplanesubclusters_U")
    pfpplanesubclusters_V_v = up.array("pfpplanesubclusters_V")
    pfpplanesubclusters_Y_v = up.array("pfpplanesubclusters_Y")
    df["shrsubclusters0"] = get_elm_from_vec_idx(pfpplanesubclusters_U_v,shr_id,0)
    df["shrsubclusters1"] = get_elm_from_vec_idx(pfpplanesubclusters_V_v,shr_id,0)
    df["shrsubclusters2"] = get_elm_from_vec_idx(pfpplanesubclusters_Y_v,shr_id,0)
    #
    # do the best we can to get the right shr2_id
    #
    shr2_id_corr = up.array('shr2_id')-1 # I think we need this -1 to get the right result
    shr2_id_appr = get_idx_from_vec_sort(-2,pfnhits_v,shr_mask)
    shr2_id = np.where((shr2_id_corr>=0)&(shr2_id_corr<df['n_showers_tot']),shr2_id_corr,shr2_id_appr)
    #
    df["shr2subclusters0"] = get_elm_from_vec_idx(pfpplanesubclusters_U_v,shr2_id,0)
    df["shr2subclusters1"] = get_elm_from_vec_idx(pfpplanesubclusters_V_v,shr2_id,0)
    df["shr2subclusters2"] = get_elm_from_vec_idx(pfpplanesubclusters_Y_v,shr2_id,0)
    df['subcluster2tmp'] = df['shr2subclusters0'] + df['shr2subclusters1'] + df['shr2subclusters2']
    #
    df["shr2_start_x"] = get_elm_from_vec_idx(shr_start_x_v,shr2_id,-9999.)
    df["shr2_start_y"] = get_elm_from_vec_idx(shr_start_y_v,shr2_id,-9999.)
    df["shr2_start_z"] = get_elm_from_vec_idx(shr_start_z_v,shr2_id,-9999.)
    df["trk1_start_x"] = get_elm_from_vec_idx(trk_start_x_v,trk_id,-9999.)
    df["trk1_start_y"] = get_elm_from_vec_idx(trk_start_y_v,trk_id,-9999.)
    df["trk1_start_z"] = get_elm_from_vec_idx(trk_start_z_v,trk_id,-9999.)
    df['tk1sh2_distance'] = np.where((df['n_showers_contained']>1)&(df['n_tracks_contained']>0),\
                                     distance(df['shr2_start_x'], df['shr2_start_y'], df['shr2_start_z'],\
                                     df['trk1_start_x'],df['trk1_start_y'],df['trk1_start_z']),\
                                     9999.)
    df['sh1sh2_distance'] = np.where(df['n_showers_contained']>1,\
                                     distance(df['shr2_start_x'], df['shr2_start_y'], df['shr2_start_z'],\
                                     df['shr_start_x'],df['shr_start_y'],df['shr_start_z']),\
                                     9999.)
    df['shr2pid'] = get_elm_from_vec_idx(trk_llr_pid_v,shr2_id,9999.)
    df['shr2_score'] = get_elm_from_vec_idx(trk_score_v,shr2_id,9999.)

    return

df_v = [mc,nue,ext,data,dirt]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    process_uproot(up,df)

In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

df_v = [mc,nue,ext,data,dirt]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    #trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    #trk_energy_proton_v = up.array('trk_energy_proton_v')
    trk_theta_proton_v  = up.array('trk_theta_v')
    trk_phi_proton_v    = up.array('trk_phi_v')
    #trk_calo_energy_y_v = up.array('trk_calo_energy_y_v')
    #trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    #trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    #trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_theta_proton_sel  = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_theta_proton_v,trk_id)])
    trk_phi_proton_sel    = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_phi_proton_v,trk_id)])
    #trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    #df['trkpid'] = trk_llr_pid_v_sel
    #df['protonenergy'] = trk_energy_proton_sel
    df['protontheta']  = trk_theta_proton_sel
    df['protonphi']    = trk_phi_proton_sel
    #df['trackcaloenergy'] = trk_calo_energy_y_sel
    #df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    #df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    #df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    #df['trkthetacos'] = np.cos(df['trk_theta'])
    ##df['FMdiffOBV'] = df['nu_flashmatch_score']-df['best_obviouscosmic_flashmatch_score']
    ##df['FMdiff'] = df['nu_flashmatch_score']-df['best_cosmic_flashmatch_score']

In [ ]:
for i,df in enumerate(df_v):
    up = uproot_v[i]
    #trk_score_v = up.array('trk_score_v')
    trk_sce_start_x_v = up.array('trk_sce_start_x_v')
    trk_sce_start_y_v = up.array('trk_sce_start_y_v')
    trk_sce_start_z_v = up.array('trk_sce_start_z_v')
    trk_sce_end_x_v = up.array('trk_sce_end_x_v')
    trk_sce_end_y_v = up.array('trk_sce_end_y_v')
    trk_sce_end_z_v = up.array('trk_sce_end_z_v')
    shr_id = up.array('shr_id')-1 # I think we need this -1 to get the right result
    #trk_score_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_score_v,shr_id)])
    trk_sce_start_x_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_start_x_v,shr_id)])
    trk_sce_start_y_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_start_y_v,shr_id)])
    trk_sce_start_z_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_start_z_v,shr_id)])
    trk_sce_end_x_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_end_x_v,shr_id)])
    trk_sce_end_y_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_end_y_v,shr_id)])
    trk_sce_end_z_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else -9999. for pidv,tid in zip(trk_sce_end_z_v,shr_id)])
    #df['shr_score_check'] = trk_score_v_sel
    df['shr_trk_sce_start_x'] = trk_sce_start_x_v_sel
    df['shr_trk_sce_start_y'] = trk_sce_start_y_v_sel
    df['shr_trk_sce_start_z'] = trk_sce_start_z_v_sel
    df['shr_trk_sce_end_x'] = trk_sce_end_x_v_sel
    df['shr_trk_sce_end_y'] = trk_sce_end_y_v_sel
    df['shr_trk_sce_end_z'] = trk_sce_end_z_v_sel   

In [ ]:
for i,df in enumerate(df_v):
    df.loc[df['shrmoliereavg'].isna(),'secondshower_Y_dot'] = 9999.
    #
    # secondshower_U/V/Y_dot need to be set to 9999. to match data ntuples produced with v08_00_00_43 or later;
    # the patch below is valid only to match data ntuples produced with v08_00_00_42.
    #
    #df.loc[df['secondshower_U_dot'].isna(),'secondshower_U_dot'] = 0.0 #for 0p please uncomment
    #df.loc[df['secondshower_V_dot'].isna(),'secondshower_V_dot'] = 0.0 #for 0p please uncomment
    df.loc[df['secondshower_Y_dot'].isna(),'secondshower_Y_dot'] = 0.0 

In [ ]:
#
for i,df in enumerate(df_v):
    up = uproot_v[i]
    df["slclnhits"] = up.array("pfnhits").sum()
    df["slclnunhits"] = up.array("pfnunhits").sum()
#

In [ ]:
df_v = [mc,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    df['shr_tkfit_nhi'] = (df['shr_tkfit_nhits_Y']*df['shr_tkfit_dedx_Y'] + df['shr_tkfit_nhits_U']*df['shr_tkfit_dedx_U'] + df['shr_tkfit_nhits_V']*df['shr_tkfit_dedx_V'])/df['shr_tkfit_nhits_tot']
    df['shr_tkfit_2cm_nhits_tot'] = (df['shr_tkfit_2cm_nhits_Y']+df['shr_tkfit_2cm_nhits_U']+df['shr_tkfit_2cm_nhits_V'])
    df['shr_tkfit_2cm_dedx_avg'] = (df['shr_tkfit_2cm_nhits_Y']*df['shr_tkfit_2cm_dedx_Y'] + df['shr_tkfit_2cm_nhits_U']*df['shr_tkfit_2cm_dedx_U'] + df['shr_tkfit_2cm_nhits_V']*df['shr_tkfit_2cm_dedx_V'])/df['shr_tkfit_2cm_nhits_tot']
    df['shr_tkfit_gap10_nhits_tot'] = (df['shr_tkfit_gap10_nhits_Y']+df['shr_tkfit_gap10_nhits_U']+df['shr_tkfit_gap10_nhits_V'])
    df['shr_tkfit_gap10_dedx_avg'] = (df['shr_tkfit_gap10_nhits_Y']*df['shr_tkfit_gap10_dedx_Y'] + df['shr_tkfit_gap10_nhits_U']*df['shr_tkfit_gap10_dedx_U'] + df['shr_tkfit_gap10_nhits_V']*df['shr_tkfit_gap10_dedx_V'])/df['shr_tkfit_gap10_nhits_tot']
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']

In [ ]:
INTERCEPT = 0.0
SLOPE = 0.83

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]
    df["reco_e_qe"] = 0.938*((df["shr_energy"]+INTERCEPT)/SLOPE)/(0.938 - ((df["shr_energy"]+INTERCEPT)/SLOPE)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
    df["extdata"] = np.zeros_like(df["shr_energy"])
data["bnbdata"] = np.ones_like(data["shr_energy"])
ext["extdata"] = np.ones_like(ext["shr_energy"])

# test calibrating dedx on Y plane
#data["shr_tkfit_gap10_dedx_Y"] = data["shr_tkfit_gap10_dedx_Y"]*1.03
#ext["shr_tkfit_gap10_dedx_Y"] = ext["shr_tkfit_gap10_dedx_Y"]*1.03
#data["shr_tkfit_2cm_dedx_Y"] = data["shr_tkfit_2cm_dedx_Y"]*1.03
#ext["shr_tkfit_2cm_dedx_Y"] = ext["shr_tkfit_2cm_dedx_Y"]*1.03
#data["shr_tkfit_dedx_Y"] = data["shr_tkfit_dedx_Y"]*1.03
#ext["shr_tkfit_dedx_Y"] = ext["shr_tkfit_dedx_Y"]*1.03

In [ ]:
# add back the cosmic category, for background only
df_v = [mc,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.2), 'category'] = 4

In [ ]:
#reset timing
df_v = [mc,nue,dirt]
for i,df in enumerate(df_v):
        df.loc[ :, 'flash_time' ] = df['flash_time' ] - 0.304

df_v = [ext]
for i,df in enumerate(df_v):
        df.loc[ :, 'flash_time' ] = df['flash_time' ] - 0.359

In [ ]:
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11


In [ ]:
# variables to be trained on
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max","trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

In [ ]:
LABELS = ['pi0','nonpi0']

if (USEBDT == True):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            #print((ls.pickle_path+'booster_%s_0304_extnumi.model' % label), booster.best_iteration)
            #booster.save_model( (ls.pickle_path+'booster_%s_0304_extnumi.model' % label) )
            mc[label+"_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nue[label+"_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ext[label+"_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            data[label+"_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVAR]),
                ntree_limit=booster.best_iteration)


In [ ]:
dfbdt_v = [mc,nue,ext,data,dirt]

# Plotting part

In [ ]:
import plotter
import importlib
importlib.reload(plotter)

In [ ]:
scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.


nue = mc.query("(abs(nu_pdg) == 12 & ccnc == 0)")
samples = {
    "mc": mc,
    "nue": nue,
    "dirt": dirt,
    "data": data,
    "ext": ext
}

scaling = 1
#scaling = 0.789
pot =  2e+20 #partial file: 8.885e+19
mcratio   = pot/2.07136e+21
dirtratio = pot/1.42143e+21
extratio  = 5268051/9199232.74 #partial file: 2347745.0/4015961.99
SPLIT = 1.0
scalingEXT = 0.98
scalingDRT = 0.35

weights = { #full sample mc = 0.38
            "mc"  : mcratio  * SPLIT * scaling,   
            "nue" : mcratio  * SPLIT * scaling,  
            "ext" : extratio * SPLIT * scalingEXT, #set0   
            "dirt": dirtratio* SPLIT * scalingDRT, #set0   
}
print (ext.shape[0])
my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and trk_len > 20'
#QUERY += ' and topological_score > 0.06'
#QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
#QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
#QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#QUERY += ' and trkpid > 0.5'
#QUERY += ' and (trkthetacos < 0.853 or trkthetacos > 0.8531)'
#QUERY += ' and bkpdg == 2212 and (trkcompl < 0.5 or trkpur < 0.5)'

print (QUERY)

In [ ]:
# nue preselection
PRESQ = 'nslice == 1'
PRESQ += ' and selected == 1'
PRESQ += ' and shr_energy_tot_cali > 0.07'
PRESQ += ' and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1)'

In [ ]:
TWOSHWRS = PRESQ
TWOSHWRS += ' and n_showers_contained >= 2'


In [ ]:
# 1eNp preselection
NPPRESQ = PRESQ
NPPRESQ += ' and n_tracks_contained > 0'

In [ ]:
# very loose box cuts
NPVLCUTQ = NPPRESQ
NPVLCUTQ += ' and CosmicIPAll3D > 10.'
NPVLCUTQ += ' and trkpid < 0.25'
NPVLCUTQ += ' and hits_ratio > 0.5'
NPVLCUTQ += ' and trkfit < 0.90'
NPVLCUTQ += ' and n_showers_contained == 1'
NPVLCUTQ += ' and tksh_distance < 10.0'
NPVLCUTQ += ' and tksh_angle > -0.9'

In [ ]:
# loose box cuts
NPLCUTQ = NPVLCUTQ
NPLCUTQ += ' and CosmicIPAll3D > 10.'
NPLCUTQ += ' and trkpid < 0.02'
NPLCUTQ += ' and hits_ratio > 0.50'
NPLCUTQ += ' and shrmoliereavg < 9'
NPLCUTQ += ' and subcluster > 4'
NPLCUTQ += ' and trkfit < 0.65'
NPLCUTQ += ' and n_showers_contained == 1'
NPLCUTQ += ' and tksh_distance < 6.0'
NPLCUTQ += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 0.5 and shr_tkfit_dedx_max < 5.5)'
NPLCUTQ += ' and secondshower_Y_nhit < 50'
NPLCUTQ += ' and tksh_angle > -0.9'

In [ ]:
NPLCUTQPlus = NPPRESQ
NPLCUTQPlus += ' and CosmicIPAll3D > 10.'
NPLCUTQPlus += ' and trkpid < 0.25'
NPLCUTQPlus += ' and hits_ratio > 0.5'
NPLCUTQPlus += ' and trkfit < 0.90'
NPLCUTQPlus += ' and tksh_distance < 10.0'
NPLCUTQPlus += ' and tksh_angle > -0.9'
NPLCUTQPlus += ' and CosmicIPAll3D > 10.'
NPLCUTQPlus += ' and trkpid < 0.02'
NPLCUTQPlus += ' and hits_ratio > 0.50'
NPLCUTQPlus += ' and shrmoliereavg < 9'
NPLCUTQPlus += ' and subcluster > 4'
NPLCUTQPlus += ' and trkfit < 0.65'
#NPLCUTQPlus += ' and n_showers_contained > 1'
NPLCUTQPlus += ' and tksh_distance < 6.0'
NPLCUTQPlus += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 0.5 and shr_tkfit_dedx_max < 5.5)'
NPLCUTQPlus += ' and secondshower_Y_nhit < 50'
NPLCUTQPlus += ' and tksh_angle > -0.9'

In [ ]:
# tight box cuts
NPTCUTQ = NPLCUTQ
NPTCUTQ += ' and CosmicIPAll3D > 30.'
NPTCUTQ += ' and CosmicDirAll3D > -0.98 and CosmicDirAll3D < 0.98'
NPTCUTQ += ' and trkpid < 0.02'
NPTCUTQ += ' and hits_ratio > 0.65'
NPTCUTQ += ' and shr_score < 0.25'
NPTCUTQ += ' and shrmoliereavg > 2 and shrmoliereavg < 10'
NPTCUTQ += ' and subcluster > 7'
NPTCUTQ += ' and trkfit < 0.70'
NPTCUTQ += ' and n_showers_contained == 1'
NPTCUTQ += ' and tksh_distance < 4.0'
NPTCUTQ += ' and trkshrhitdist2 < 1.5'
NPTCUTQ += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 1.0 and shr_tkfit_dedx_max < 3.8)'
NPTCUTQ += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
NPTCUTQ += ' and secondshower_Y_nhit < 30'
NPTCUTQ += ' and tksh_angle > -0.9 and tksh_angle < 0.70'

In [ ]:
# OLD loose box cuts
OLCUTQ = PRESQ
OLCUTQ += ' and CosmicIP > 20.'
OLCUTQ += ' and trkpid < 0.1'
OLCUTQ += ' and hits_ratio > 0.5'
OLCUTQ += ' and shr_score < 0.30'
OLCUTQ += ' and n_showers_contained == 1'
OLCUTQ += ' and tksh_distance < 6.0'
OLCUTQ += ' and shr_tkfit_2cm_dedx_avg < 4.0'
OLCUTQ += ' and tksh_angle > -0.9'
# OLD tight box cuts
OTCUTQ = OLCUTQ
OTCUTQ += ' and tksh_distance < 3.5'
OTCUTQ += ' and (shr_tkfit_2cm_dedx_Y > 0 and shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
OTCUTQ += ' and (shr_tkfit_gap10_dedx_Y > 0 and shr_tkfit_gap10_dedx_Y < 4.5 and shr_tkfit_gap10_dedx_U < 4.5 and shr_tkfit_gap10_dedx_V < 4.5)'
OTCUTQ += ' and tksh_angle > -0.9 and tksh_angle < 0.75'
OTCUTQ += ' and shrmoliereavg > 2 and shrmoliereavg < 9'
OTCUTQ += ' and trkpid < -0.02'
OTCUTQ += ' and n_showers_contained == 1'
OTCUTQ += ' and shr_score < 0.275'
OTCUTQ += ' and hits_ratio > 0.60'
OTCUTQ += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
OTCUTQ += ' and (trkfit < 0.45 or subcluster > 6)'

In [ ]:
# BDT cuts
# 0304 extnumi, pi0 and nonpi0
BDTCQ = NPPRESQ
BDTCQ += ' and pi0_score > 0.67 and nonpi0_score > 0.70'

In [ ]:
myPre = "nslice == 1 and selected == 1 and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and n_tracks_contained > 0"

In [ ]:
# loose box cuts
NPLCUTQ2 = NPPRESQ
NPLCUTQ2 += ' and CosmicIPAll3D > 10.'
NPLCUTQ2 += ' and hits_ratio > 0.5'
NPLCUTQ2 += ' and trkfit < 0.90'
NPLCUTQ2 += ' and n_showers_contained == 1'
NPLCUTQ2 += ' and tksh_distance < 10.0'
NPLCUTQ2 += ' and tksh_angle > -0.9'
NPLCUTQ2 += ' and CosmicIPAll3D > 10.'
#NPLCUTQ2 += ' and trkpid < 0.02'
NPLCUTQ2 += ' and hits_ratio > 0.50'
NPLCUTQ2 += ' and shrmoliereavg < 9'
NPLCUTQ2 += ' and subcluster > 4'
NPLCUTQ2 += ' and trkfit < 0.65'
NPLCUTQ2 += ' and n_showers_contained == 1'
NPLCUTQ2 += ' and tksh_distance < 6.0'
NPLCUTQ2 += ' and (shr_tkfit_nhits_tot > 1 and shr_tkfit_dedx_max > 0.5 and shr_tkfit_dedx_max < 5.5)'
NPLCUTQ2 += ' and secondshower_Y_nhit < 50'
NPLCUTQ2 += ' and tksh_angle > -0.9'

In [ ]:
SCORECUT = 0.5 # 0.75 #75 # max track score                                                                                                                                                                        
DVTX = 3.0 # 3. # distance from vertex of each shower                                                                                                                                                              
VTXDOT = 0.8 # dot product between each shower's direction and the vtx -> shr start vector                                                                                                                         
EMIN1 =  60 #150 #60   # leading photon min energy                                                                                                                                                                        
EMIN2 =  40 #100 #40   # subleading photon min energy                                                                                                                                                          
GAMMADOT = 0.94 #0 #0.94 # max dot product between showres                                                                                                                                                                  
DEDXCUT = 1.0 # MeV/cm cut on leading shower only                                                                                                                                                                  
PI0SEL = 'nslice == 1'
PI0SEL += ' & pi0_shrscore1 < %f & pi0_shrscore2 < %f'%(SCORECUT,SCORECUT)
PI0SEL += '& pi0_dot1  > %f & pi0_dot2 > %f '%(VTXDOT,VTXDOT)
PI0SEL += ' & pi0_radlen1 > %f & pi0_radlen2 > %f & pi0_gammadot < %f '%(DVTX,DVTX,GAMMADOT)
PI0SEL += ' & pi0_energy1_Y > %f & pi0_energy2_Y > %f'%(EMIN1,EMIN2)
#PI0SEL += ' and (filter_pi0 == 1 or bnbdata==1 or extdata==1)'                                                                                                                                                    
#PI0SEL += ' and (filter_pi0 == 1)'                                                                                                                                                                                
PI0SEL += ' and pi0_dedx1_fit_Y >= %f'%DEDXCUT

In [ ]:
NUMUOnTheFly = 'nslice == 1 and topological_score > 0.06 '
NUMUOnTheFly += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
NUMUOnTheFly += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
NUMUOnTheFly += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
NUMUOnTheFly += ' and trk_score > 0.8 and trk_len > 10 and trk_distance < 4'

In [ ]:
# Define actual selection
# QUERY = NPTCUTQ #NPTCUTQ
#QUERY = NPPRESQ
#QUERY = NPLCUTQ #NPTCUTQ NPLCUTQ
#QUERY = NPPRESQ 
QUERY = NPLCUTQ +" and "+ BDTCQ #PI0SEL#NUMUOnTheFly + ' and n_showers_contained == 1'  #PI0SEL#NPLCUTQ2 #NPLCUTQ #NPPRESQ#'nslice ==1 ' #NPPRESQ #PRESQ
# in case you do not want to look at the data
#QUERY += ' and bnbdata==0'
# in case you want to apply the CRT veto
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'

In [ ]:
print(NPLCUTQPlus)

In [ ]:
NN = NPVLCUTQ #NPLCUTQ #'nslice == 1 and topological_score > 0.06 '
#NN += ' and n_tracks_contained == 2 ' #and N_recoMuons == 0'
NN += ' and N_recoProtons == 2 ' #and N_recoMuons == 0'
#NN += ' and n_showers_contained == 1'


In [ ]:
#QUERY = NN #"nslice >= 0" #BDTCQ #+" and "+NPLCUTQ
QUERY = BDTCQ +" and "+NPLCUTQ
#QUERY = NPLCUTQ
import matplotlib.pyplot as plt
import matplotlib
# v = "$\cos\left(\theta_{\mu}\right)$"

#QUERY += " & pi0truth_elec_etot<15 & npi0 == 0 & ccnc == 0"
#QUERY += " & muon_e>0.3 & npi0 == 0 & ccnc == 0"
#QUERY = "(muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category == 2"
#QUERY += " & ~((muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category != 5)"


VARIABLE, BINS, RANGE, XTIT = 'reco_e', 20, (0.15,4.15), r"Reconstructed Energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_y',12,(-120,120),"y [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'flash_time',4,(0,20.),"flash time"


#VARIABLE, BINS, RANGE, XTIT = 'pi0_mass_Y',100,(0,1000.),"pi0 mass Y"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_dedx1_fit_Y',25,(0,10.),"pi0 dedx1 fit Y"
#VARIABLE, BINS, RANGE, XTIT = 'trk_len',50,(0,400.),"track length"

#VARIABLE, BINS, RANGE, XTIT = 'pi0_gammadot',10,(-1.,1.0),r" pi0 gammadot "
#VARIABLE, BINS, RANGE, XTIT = 'topological_score',25,(0,1),r"topological score"
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_x',20,(-10,260),"x"
#VARIABLE, BINS, RANGE, XTIT = 'nonpi0_score',10,(0.5,1.5),"non pi0 score"

#VARIABLE, BINS, RANGE, XTIT = 'n_showers_contained',10,(0.,10.),r"n showers contained"
#VARIABLE, BINS, RANGE, XTIT = 'pi0truth_elec_etot',20,(0.,60),r"Michel Energy [GeV]"
#QUERY = PI0SEL #NPLCUTQ2#BDTCQ+" and "+NPLCUTQ
#fig, ax1, ax2 = my_plotter.plot_variable(
fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query= QUERY ,#BDTCQ,#NPLCUTQ, #QUERY, #'nslice == 1', #QUERY, #+" and category != 111",
    kind="event_category",
    #kind="sample",
    #kind="interaction",
    #kind='particle_pdg',
    draw_sys=True, #False,
    stacksort=3,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)[0:3]

#print("Profile likelihood: %.2f sigma @ 1.01e21 POT" % my_plotter.significance_likelihood)
#print("s/sqrt(b): %.2f sigma @ 1.01e21 POT" % my_plotter.significance)

#ax1.set_ylim(0,40)
#ax1.set_yscale("log")
#ax1.set_ylim(0,240)
#ax1.set_yscale('log')
#ax1.set_ylim(0,ax1.get_ylim()[1]*1.25)
#ax2.set_ylim(0.,2.)

#ax1.set_title('NuMI Run 1 \n1eNp BDT Sel - $\pi^0$ Scaling: [1-0.4 E$_{\pi^0}$ ], Stat only',loc='left')
ax1.set_title('NuMI 2 protons Selection',loc='left')
fig.tight_layout()
#fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/"+VARIABLE+".pdf")
#fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/Pi0/"+VARIABLE+"_noScale.pdf")

In [ ]:
print('%s %.4f ' % (VARIABLE, my_plotter.stats['pvaluestatonly']))

In [ ]:
#VARIABLE, BINS, RANGE, XTIT = 'reco_e', 10, (0.15,2.15), r"Reconstructed Energy [GeV]"
VARIABLE, BINS, RANGE, XTIT = 'subcluster',15,(0,60),r"N sub-clusters in shower"


fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query= QUERY ,
    kind="event_category",
    draw_sys=False,
    stacksort=3,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)[0:3]

ax1.set_title('NuMI Run 1 \n1eNp BDT Sel - $\pi^0$ Scaling: [1-0.4 E$_{\pi^0}$ ], Stat Only',loc='left')
#fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/BDTSel/"+VARIABLE+".pdf")

In [ ]:
print('%s %.4f ' % (VARIABLE, my_plotter.stats['pvaluestatonly']))

In [ ]:
print(NPLCUTQ)
print()
print(BDTCQ)
print()
print(myPre)

In [ ]:
#print(data.query(QUERY)[["run","sub","evt","reco_e"]])
#QUERY += " and shr_theta > 1.5"
print(data.query(QUERY)[["run","sub","evt"]])
f = data.query(QUERY)[["run","sub",]]
#tablevars = ["run","sub","evt","reco_e","pi0_score","nonpi0_score"]
#tablevars = ["run","sub","evt"]
#tablevars+=TRAINVAR
#print(tablevars)
#data.query(PRESQ+" & run==14822")[tablevars]
#data.query("run==5151")[tablevars]

query4SamWeb = f.apply(lambda x: str(x['run'])+'.'+str(x['sub'])+',', axis=1).sum()
print("defname: numi_uboone_run1_beamon_offset1_mcc9_reco2_v08_00_00_28_beam_good and run_number "+query4SamWeb)


In [ ]:
mc.query(QUERY+"& ccnc==0 & nu_pdg==14 & mcf_np0==0")[["run","sub","evt","reco_e","mcf_lep_e","mcf_mcshr_elec_etot"]]

In [ ]:
#PI0SEL #QUERY = BDTCQ+" and "+NPLCUTQ#+" and reco_e < 0.5" #NPLCUTQ2# BDTCQ #BDTCQ+" and "+NPLCUTQ
QUERY = BDTCQ +" and "+NPLCUTQ #NPLCUTQ#+" and reco_e < 0.5" #NPLCUTQ2# BDTCQ #BDTCQ+" and "+NPLCUTQ

'''
print(QUERY)

scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.


nue = mc.query("(abs(nu_pdg) == 12 & ccnc == 0)")
samples = {
    "mc": mc,
    "nue": nue,
    "dirt": dirt,
    "data": data,
    "ext": ext
}

scaling = 1
pot = 8.885e+19 #9.789e+19 
mcratio   = pot/2.07136e+21
dirtratio = pot/1.42143e+21
extratio  = 2347745.0/4015961.99
SPLIT = 1.0
scalingEXT = 0.98
scalingDRT = 0.45

weights = { #full sample mc = 0.38
            "mc"  : mcratio  * SPLIT * scaling,   
            "nue" : mcratio  * SPLIT * scaling,  
            "ext" : 1, #extratio * SPLIT * 1, #set0   
            "dirt": dirtratio* SPLIT * 1, #set0   
}

my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)
'''
QUERY = BDTCQ +" and "+NPLCUTQ #+" and shr_energy_tot_cali < 0.5"
plots = []

plots.append(['reco_nu_vtx_sce_x',10,(0,260),"vtx x [cm]"])
plots.append(['reco_nu_vtx_sce_y',10,(-120,120),"vtx y [cm]"])
plots.append(['reco_nu_vtx_sce_z',10,(0,1030),"vtx z [cm]"])
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_x',5,(0,260),"vtx x [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_y',5,(-120,120),"vtx y [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'reco_nu_vtx_sce_z',5,(0,1030),"vtx z [cm]"

plots.append(['reco_e', 10, (0.15,2.15), 'Reconstructed Energy [GeV]'])
plots.append(['shr_score',20,(0,0.5),"shr score"])
plots.append(['tksh_distance',10,(0,5),"tksh distance [cm]"])
plots.append(['tksh_angle',10,(-1,1),"cos(tksh angle)"])
plots.append(['shr_tkfit_dedx_max',20,(0,10),"shr tkfit dE/dx (best-plane, 0-4 cm) [MeV/cm]"])
plots.append(['trkfit',10,(0,1),"Fraction of Track-fitted points"])
plots.append(['trkpid',10,(-1,1),"track LLR PID"])
plots.append(['shrmoliereavg',10,(0,25),"average Moliere angle [degrees]"])
plots.append(['subcluster',15,(0,60),"N sub-clusters in shower"])
plots.append(['hits_ratio',20,(0,1),"shower hits/all hits"])
plots.append(['trkshrhitdist2',16,(-1,15),"2D trk-shr distance (Y)"])
plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
plots.append(['secondshower_Y_vtxdist',20,(0.,400),"vtx dist 2nd shower (Y)"])
plots.append(['anglediff_Y',10,(0,360),"angle diff 1st-2nd shower (Y) [degrees]"])
plots.append(['CosmicIPAll3D',10,(0,200),"CosmicIPAll3D [cm]"])
plots.append(['CosmicDirAll3D',10,(-1,1),"cos(CosmicDirAll3D)"])
plots.append(['reco_nu_vtx_sce_y',15,(-150,150),"reco nu vtx y [cm]"])
             
#plots.append(['shr_tkfit_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_max',20,(0,10),"shr tkfit dE/dx (best-plane, 0-4 cm) [MeV/cm]"])
#plots.append(['n_showers_contained',10,(0,10),"n showers contained"])
plots.append(['n_tracks_contained',5,(0,5),"n tracks contained"])
plots.append([ 'trk_energy_tot',12,(0,0.6), "Total kinetic energy in protons [GeV]"])
plots.append(['protonenergy',10,(0.,0.5),"Proton Kinetic Energy [GeV]"])
plots.append(['trk_theta',10,(0,3.0),"Track theta"])
plots.append(['shr_theta',10,(0,3.0),"Shower theta"])
plots.append(['pt',10,(0,2.),"Pt [GeV]"])
plots.append(['shr_phi',10,(-3,3),"Shower phi"])
#plots.append(['reco_nu_vtx_sce_x',5,(0,260),"vtx x [cm]"])
#plots.append(['reco_nu_vtx_sce_y',5,(-120,120),"vtx y [cm]"])
#plots.append(['reco_nu_vtx_sce_z',5,(0,1030),"vtx z [cm]"])

#plots.append(['reco_nu_vtx_sce_z',20,(0,1100),"z"])
#plots.append(['reco_e_rqe',20,(0,2),"rqe"])
#plots.append(['shrmoliererms',20,(0,5000),"RMS Moliere angle [degrees]"])
plots.append(['shr_energy_tot_cali',10,(0,1),"shr energy (calibrated) [GeV]"])
#plots.append(['contained_fraction',20,(0.,1),"contained fraction"])
#plots.append(['hits_y',20,(0.,1000),"N hits Y plane"])
#plots.append(['pi0_mass_Y',25,(-50.,200),"pi0 mass [Mev]"])
#plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])
if USEBDT:
    #plots = []
    #plots.append(['pi0_score',20,(0.9,1),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',20,(0.9,1),"non-$\pi^0$ BDT response"])
    #plots.append(['pi0_score',10,(0.,0.5),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',10,(0.,0.5),"non-$\pi^0$ BDT response"])
    plots.append(['pi0_score',10,(0,1.0),"$\pi^0$ BDT response"])
    plots.append(['nonpi0_score',10,(0,1.0),"non-$\pi^0$ BDT response"])

plots.append([ 'protontheta',10,(0,3.0), "Leading proton theta [rad]"])
plots.append([ 'protonphi',10,(-3.0,3.0), "Leading proton phi [rad]"])

    #plots = []
#plots.append(['tksh_distance',20,(0,10),"tksh distance [cm]"])

#plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
#plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
#plots.append(['anglediff_Y',20,(0,350),"angle diff 1st-2nd shower (Y) [degrees]"])
#plots.append(['secondshower_Y_vtxdist',20,(0.,200),"vtx dist 2nd shower (Y)"])
f = open("NuMIPValuesStatOnly_1eNp.txt", "a")
for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    #ax1.set_ylim(0,ax1.get_ylim()[1]*1.4)
    #ax1.set_yscale("log")
    #fig.show()
    #ax1.set_title('NuMI Run 1 \n1eNp Loose Box Cuts - No pi0 scaling',loc='left')
    ax1.set_title('NuMI Run 1 \n1eNp BDT Sel. - $\pi^0$ Scaling: [1-0.4 E$_{\pi^0}$ ], Stat Only',loc='left')
    fig.tight_layout()
    #fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/"+VARIABLE+".pdf")
    fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/BDTFinal/"+VARIABLE+".pdf")
    #fig.savefig(ls.plots_path+""+VARIABLE+"_highscore.pdf")
    f.write('%s %.4f ' % (VARIABLE, my_plotter.stats['pvaluestatonly'])+"\n")
    #break
f.close()

In [ ]:
QUERY == BDTCQ+" and "+NPLCUTQ+" and reco_e < 0.5"
print(QUERY)
f = data.query(QUERY)[["run","sub"]]#.to_string(index=False)
query4SamWeb = f.apply(lambda x: str(x['run'])+'.'+str(x['sub'])+',', axis=1).sum()
print("defname: numi_uboone_run1_beamon_offset1_mcc9_reco2_v08_00_00_28_v0 and run_number "+query4SamWeb)

f = data.query(QUERY)[["run","sub","evt","reco_e"]]
print (f)

In [ ]:
scaleFlat = 2
df_v = [mc,nue,dirt]
#removeGenieTune = False
for i,df in enumerate(df_v):
    if scaleFlat == 1:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 
        df.loc[ df['npi0'] > 0, 'weightSplineTimesTune' ] =  (df['weightSplineTimesTune'] * 0.759)
    elif scaleFlat == 2:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 
        pi0emax = 0.6
        df.loc[ (df['pi0_e'] > 0.1) & (df['pi0_e'] < pi0emax) , 'weightSplineTimesTune'] = df['weightSplineTimesTune']*(1.-0.4*df['pi0_e'])
        df.loc[ (df['pi0_e'] > 0.1) & (df['pi0_e'] >= pi0emax), 'weightSplineTimesTune'] = df['weightSplineTimesTune']*(1.-0.4*pi0emax)
    elif scaleFlat == 0:
        df['weightSplineTimesTune'] = df[ 'currentweight' ] 


In [ ]:
QUERY = BDTCQ +" and "+NPLCUTQ
plots = []
plots.append(['shr_theta',100,(0.,3.0),"Shower theta"])
plots.append(['shr_phi',10,(-3,3),"Shower phi"])
'''
plots.append(['reco_e', 10, (0.15,2.15), 'Reconstructed Energy [GeV]'])
plots.append(['shr_score',20,(0,0.5),"shr score"])
plots.append(['tksh_distance',5,(0,5),"tksh distance [cm]"])
plots.append(['tksh_angle',10,(-1,1),"cos(tksh angle)"])
plots.append(['shr_tkfit_dedx_max',15,(0,10),"shr tkfit dE/dx (best-plane, 0-4 cm) [MeV/cm]"])
plots.append(['trkfit',10,(0,1),"Fraction of Track-fitted points"])
plots.append(['trkpid',10,(-1,1),"track LLR PID"])
plots.append(['shrmoliereavg',10,(0,25),"average Moliere angle [degrees]"])
plots.append(['subcluster',15,(0,60),"N sub-clusters in shower"])
plots.append(['hits_ratio',20,(0,1),"shower hits/all hits"])
plots.append(['trkshrhitdist2',16,(-1,15),"2D trk-shr distance (Y)"])
plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
plots.append(['secondshower_Y_vtxdist',20,(0.,400),"vtx dist 2nd shower (Y)"])
plots.append(['anglediff_Y',10,(0,360),"angle diff 1st-2nd shower (Y) [degrees]"])
plots.append(['CosmicIPAll3D',10,(0,200),"CosmicIPAll3D [cm]"])
plots.append(['CosmicDirAll3D',10,(-1,1),"cos(CosmicDirAll3D)"])
plots.append(['reco_nu_vtx_sce_y',15,(-150,150),"reco nu vtx y [cm]"])
             
#plots.append(['shr_tkfit_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_2cm_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-2 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_dedx_avg',20,(0,10),"shr tkfit dE/dx (avg, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_gap10_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 1-5 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_nhits_tot',20,(0,20),"shr tkfit nhits (tot, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_max',20,(0,10),"shr tkfit dE/dx (best-plane, 0-4 cm) [MeV/cm]"])
#plots.append(['n_showers_contained',10,(0,10),"n showers contained"])
plots.append(['n_tracks_contained',5,(0,5),"n tracks contained"])
plots.append([ 'trk_energy_tot',12,(0,0.6), "Total kinetic energy in protons [GeV]"])
plots.append([ 'protontheta',10,(0,3.0), "Leading proton theta [rad]"])
#plots.append([ 'protonphi',10,(-3.0,3.0), "Leading proton phi [rad]"])
plots.append(['protonenergy',10,(0.,0.5),"Proton Kinetic Energy [GeV]"])
plots.append(['trk_theta',10,(0,3.0),"Track theta"])
plots.append(['shr_theta',10,(0,3.0),"Shower theta"])
plots.append(['pt',10,(0,2.),"Pt [GeV]"])
plots.append(['shr_phi',10,(-3,3),"Shower phi"])
#plots.append(['reco_nu_vtx_sce_z',20,(0,1100),"z"])
#plots.append(['reco_e_rqe',20,(0,2),"rqe"])
#plots.append(['shrmoliererms',20,(0,5000),"RMS Moliere angle [degrees]"])
plots.append(['shr_energy_tot_cali',10,(0,1),"shr energy (calibrated) [GeV]"])
#plots.append(['contained_fraction',20,(0.,1),"contained fraction"])
#plots.append(['hits_y',20,(0.,1000),"N hits Y plane"])
#plots.append(['pi0_mass_Y',25,(-50.,200),"pi0 mass [Mev]"])
#plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])
if USEBDT:
    #plots = []
    #plots.append(['pi0_score',20,(0.9,1),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',20,(0.9,1),"non-$\pi^0$ BDT response"])
    #plots.append(['pi0_score',10,(0.,0.5),"$\pi^0$ BDT response"])
    #plots.append(['nonpi0_score',10,(0.,0.5),"non-$\pi^0$ BDT response"])
    plots.append(['pi0_score',10,(0,1.0),"$\pi^0$ BDT response"])
    plots.append(['nonpi0_score',10,(0,1.0),"non-$\pi^0$ BDT response"])
'''
#plots = []
#plots.append(['tksh_distance',20,(0,10),"tksh distance [cm]"])

#plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
#plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
#plots.append(['anglediff_Y',20,(0,350),"angle diff 1st-2nd shower (Y) [degrees]"])
#plots.append(['secondshower_Y_vtxdist',20,(0.,200),"vtx dist 2nd shower (Y)"])
f = open("NuMIPValuesStatOnly_1eNp.txt", "a")
for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    #ax1.set_ylim(0,ax1.get_ylim()[1]*1.4)
    #ax1.set_yscale("log")
    #fig.show()
    #ax1.set_title('NuMI Run 1 \n1eNp Loose Box Cuts - No pi0 scaling',loc='left')
    ax1.set_title('NuMI Run 1 \n1eNp BDT Sel. - $\pi^0$ Scaling: [1-0.4 E$_{\pi^0}$ ], Stat Only',loc='left')
    fig.tight_layout()
    #fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/"+VARIABLE+".pdf")
    fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/1eNp/BDTSel_allStat/"+VARIABLE+".pdf")
    #fig.savefig(ls.plots_path+""+VARIABLE+"_highscore.pdf")
    f.write('%s %.4f ' % (VARIABLE, my_plotter.stats['pvaluestatonly'])+"\n")
    #break
f.close()

In [ ]:


QUERY = BDTCQ
plots = []

plots.append(['pi0_dedx1_fit_Y',20,(0,10),"pi0 dedx1 fit Y [MeV/cm]"])
plots.append(['pi0_dedx2_fit_Y',20,(0,10),"pi0 dedx2 fit Y [MeV/cm]"])
plots.append(['tksh_distance',20,(0,60),"tksh distance [cm]"])
plots.append(['pi0_energy1_Y',26,(0.,650),"pi0 energy1 [Mev]"])    
plots.append(['pi0_energy2_Y',26,(0.,650),"pi0 energy2 [Mev]"])    
plots.append(['pi0_mass_Y',25,(0.,500),"pi0 mass [Mev]"])
plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])

#f.write(" Pi0 \n")
for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    ax1.set_title('NuMI Run 1 \nPi0 Sel - No pi0 scaling',loc='left')
    fig.tight_layout()
    #fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/Pi0/"+VARIABLE+"_noScale.pdf")
    print('%s %.4f ' % (VARIABLE, my_plotter.stats['pvaluestatonly'])+"\n")

In [ ]:
scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.


nue = mc.query("(abs(nu_pdg) == 12 & ccnc == 0)")
samples = {
    "mc": mc,
    "nue": nue,
    "dirt": dirt,
    "data": data,
    "ext": ext
}

#scaling = 1
scaling = 0.759
pot = 8.885e+19 #9.789e+19 
mcratio   = pot/2.07136e+21
dirtratio = pot/1.42143e+21
extratio  = 2347745.0/4015961.99
SPLIT = 1.0
scalingEXT = 0.98
scalingDRT = 0.45

weights = { #full sample mc = 0.38
            "mc"  : mcratio  * SPLIT * scaling,   
            "nue" : mcratio  * SPLIT * scaling,  
            "ext" : extratio * SPLIT * 1, #set0   
            "dirt": dirtratio* SPLIT * 1, #set0   
}

my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)
QUERY = PI0SEL
plots = []

plots.append(['pi0_dedx1_fit_Y',20,(0,10),"pi0 dedx1 fit Y [MeV/cm]"])
plots.append(['pi0_dedx2_fit_Y',20,(0,10),"pi0 dedx2 fit Y [MeV/cm]"])
plots.append(['tksh_distance',20,(0,60),"tksh distance [cm]"])
plots.append(['pi0_energy1_Y',26,(0.,650),"pi0 energy1 [Mev]"])    
plots.append(['pi0_energy2_Y',26,(0.,650),"pi0 energy2 [Mev]"])    
plots.append(['pi0_mass_Y',25,(0.,500),"pi0 mass [Mev]"])
plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])

for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    ax1.set_title('NuMI Run 1 \nPi0 Sel - 0.759 pi0 scaling',loc='left')
    fig.tight_layout()
    fig.savefig("/Users/elenag/Desktop/NuMI/NuMI4PeLEE/Sidebands/Figures/NuMI/Pi0/"+VARIABLE+"_Scale.pdf")

In [ ]:
PRESQ = 'nslice == 1'
PRESQ += ' and selected == 1'
PRESQ += ' and shr_energy_tot_cali > 0.07'
PRESQ += ' and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1)'

ZPPRESEL = PRESQ
ZPPRESEL += ' and n_tracks_contained == 0'
ZPPRESEL_one_shower = ZPPRESEL + 'and n_showers_contained == 1'

ZPLOOSESEL = ZPPRESEL
ZPLOOSESEL += ' and n_showers_contained == 1'
ZPLOOSESEL += ' and CosmicIPAll3D > 10.'
ZPLOOSESEL += ' and CosmicDirAll3D > -0.9 and CosmicDirAll3D < 0.9'
ZPLOOSESEL += ' and shrmoliereavg < 15'
ZPLOOSESEL += ' and subcluster > 4'
ZPLOOSESEL += ' and trkfit < 0.65'
ZPLOOSESEL += ' and secondshower_Y_nhit < 50'
ZPLOOSESEL += ' and shr_trk_sce_start_y > -100 and shr_trk_sce_start_y < 80'
ZPLOOSESEL += ' and shr_trk_sce_end_y > -100 and shr_trk_sce_end_y < 100 '
ZPLOOSESEL += ' and shr_trk_len < 300.'
ZPLOOSESEL += ' and (n_tracks_tot == 0 or (n_tracks_tot>0 and tk1sh1_angle_alltk>-0.9))'
ZPLOOSESEL += ' and n_tracks_contained == 0'

ZPBDT = ZPLOOSESEL
ZPBDT += ' and bkg_score >0.85'

QUERY = ZPBDT

In [ ]:
scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.


nue = mc.query("(abs(nu_pdg) == 12 & ccnc == 0)")
samples = {
    "mc": mc,
    "nue": nue,
    "dirt": dirt,
    "data": data,
    "ext": ext
}

scaling = 1
#scaling = 0.759
pot = 8.885e+19 #9.789e+19 
mcratio   = pot/2.07136e+21
dirtratio = pot/1.42143e+21
extratio  = 2347745.0/4015961.99
SPLIT = 1.0
scalingEXT = 0.98
scalingDRT = 0.45

weights = { #full sample mc = 0.38
            "mc"  : mcratio  * SPLIT * scaling,   
            "nue" : mcratio  * SPLIT * scaling,  
            "ext" : extratio * SPLIT * scalingEXT, #set0   
            "dirt": dirtratio* SPLIT * scalingDRT, #set0   
}

my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)
plots = []

plots.append(['reco_e',20,(0,10),"reco E [MeV]"])

for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        stacksort=3,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )[0:3]
    ax1.set_title('NuMI Run 1',loc='left')
    fig.tight_layout()